In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
#Load Amazon Data into Spark Dataframe
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_game_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
video_game_df.show()

In [ ]:
#Filter by votes
filtered_video_game_df = video_game_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verfied_purchase"])
filtered_video_game_df.show()

In [ ]:
#filter for greater than 20 votes
total_votes_df = filtered_video_game_df.filter(filtered_video_game_df["total_votes"] >= 20)
total_votes_df.show()

In [ ]:
#filter for greater than 50% helpful vote percentage
helpful_votes_df = total_votes_df.filter(total_votes_df["helpful_votes"]/total_votes_df["total_votes"] >= 0.5)
helpful_votes_df.show()

In [ ]:
#Describe the Stats
paid_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
paid_df.describe().show(10)

In [ ]:
unpaid_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
unpaid_df.describe().show(10)

In [ ]:
#Determine the five star reveiw among non-vine reviews
paid_five_star_number = paid_df[paid_df("star_rating") == 5].couunt()
paid_number = paid_df.count()
percentage_five_star_vine = round(float(paid_five_star_number) / float(paid_number), 4)
percentage_five_star_vine

In [ ]:
print(f"Number of Paid Reviews {paid_number}")
print(f"Number of Paid Five Star Reviews {paid_five_star_number}")
print(f"Percentage of paid reveiws that are five stars {percentage_five_star_vine * 100}%")

In [ ]:
unpaid_five_star_number = unpaid_df[unpaid_df("star_rating") == 5].couunt()
unpaid_number = unpaid_df.count()
percentage_five_star_vine = round(float(unpaid_five_star_number) / float(unpaid_number), 4)
percentage_five_star_vine

In [ ]:
print(f"Number of UnPaid Reviews {unpaid_number}")
print(f"Number of UnPaid Five Star Reviews {unpaid_five_star_number}")
print(f"Percentage of paid reveiws that are five stars {percentage_five_star_vine * 100}%")